In [1]:
import json
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import sys
sys.path.insert(0, "C:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\rag-pipeline")
sys.path.insert(0, "C:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\llm-judge-context-dataset")
print(sys.path)
from rag_pipeline import RAG

with open('C:/Users/jaide/Documents/CodingProjects/NeuroSymbolicAI/llm-judge-context-dataset/dataset.json') as f:
    d = json.load(f)
    print(d)
d = d['data']

c:\Users\jaide\Documents\CodingProjects\NeuroSymbolicAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['C:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\llm-judge-context-dataset', 'C:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\rag-pipeline', 'C:\\Python312\\python312.zip', 'C:\\Python312\\DLLs', 'C:\\Python312\\Lib', 'C:\\Python312', 'c:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\.venv', '', 'c:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\.venv\\Lib\\site-packages', 'c:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\.venv\\Lib\\site-packages\\win32', 'c:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\.venv\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jaide\\Documents\\CodingProjects\\NeuroSymbolicAI\\.venv\\Lib\\site-packages\\Pythonwin']
{'data': [{'id': 6401197308716204890, 'prompt': 'which method of forecasting uses averages to predict future weather?', 'canAnswer': True, 'context': ['Seasonality is a characteristic of a time series in which the data experiences regular and predictable changes whic

In [2]:
rag = RAG()

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from C:\Users\jaide\.cache\huggingface\hub\models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF\snapshots\3a6fbf4a41a1d52e415a4958cde6856d34b2db93\mistral-7b-instruct-v0.2.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_mo

In [4]:
outputs = []
evaluations = []
for entry in d:
    question = entry["prompt"]
    docs = entry['context']
    id = entry['id']
    collection = "collection"+str(id)
    canAnswer = entry['canAnswer']
    rag.add_documents(collection, docs)
    output = rag.rag_query(collection, question + " If the question is not related to the context or the context does not contain the answer to the question, then say 'I don't know'.")
    prompt = f"""[System]
    Please act as an impartial judge and evaluate the quality of the response provided by an 
    AI assistant to the user question displayed below. Your evaluation should consider factors 
    such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of 
    the response. Begin your evaluation by providing a short explanation. Be as objective as 
    possible. After providing your explanation, please rate the response on a scale of 1 to 10 
    by strictly following this format: "[[rating]]", for example: "Rating: [[5]]. Also, the Assistant should not answer certain questions, denoted by [Can Answer]. If this parameter is false, then give the Assistant a rating of 1".
    
    [Can Answer]
    {canAnswer}

    [Question]
    {question}
    [The Start of Assistant’s Answer]
    {output}
    [The End of Assistant’s Answer]"""
    eval_output = rag.query(prompt, 512)
    outputs.append(output)
    evaluations.append(eval_output)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18133.85 ms
llama_print_timings:      sample time =      49.38 ms /   225 runs   (    0.22 ms per token,  4556.22 tokens per second)
llama_print_timings: prompt eval time =   44645.29 ms /   164 tokens (  272.23 ms per token,     3.67 tokens per second)
llama_print_timings:        eval time =   36468.58 ms /   224 runs   (  162.81 ms per token,     6.14 tokens per second)
llama_print_timings:       total time =   54553.12 ms /   388 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   18133.85 ms
llama_print_timings:      sample time =      27.99 ms /   125 runs   (    0.22 ms per token,  4465.24 tokens per second)
llama_print_timings: prompt eval time =   40671.29 ms /   442 tokens (   92.02 ms per token,    10.87 tokens per second)
llama_print_timings:        eval time =   21919.13 ms /   124 runs   (  176.77 ms per token,     5.66 tokens per second)
llama_print_timings:       to

In [5]:
for i in range(len(outputs)):
    print("Output:", outputs[i])
    print("Evaluation:",evaluations[i])
    print("_________________________________________________________________________________________________________________________")


Output: The provided context discusses methods for calculating seasonal indices in time series forecasting, specifically using Moving Average or Exponential Smoothing. However, it does not directly state that these methods are used to predict future weather based on averages.

                To answer the question, there are different types of time series forecasting methods that use averages as a basis for their predictions. One such method is Simple Moving Average (SMA), where the forecast for a given time period is the average of a certain number of previous periods. Another method is Autoregressive Integrated Moving Average (ARIMA), which uses historical data and mathematical formulas to predict future values based on past trends, including seasonality, and can include averages as part of its model.

                In the context given, no specific forecasting method related to weather prediction is mentioned that is solely based on averages. I'd suggest considering other sources

In [ ]:
"""Output: The provided context discusses methods for calculating seasonal indices in time series forecasting, specifically using Moving Average or Exponential Smoothing. However, it does not directly state that these methods are used to predict future weather based on averages.

                To answer the question, there are different types of time series forecasting methods that use averages as a basis for their predictions. One such method is Simple Moving Average (SMA), where the forecast for a given time period is the average of a certain number of previous periods. Another method is Autoregressive Integrated Moving Average (ARIMA), which uses historical data and mathematical formulas to predict future values based on past trends, including seasonality, and can include averages as part of its model.

                In the context given, no specific forecasting method related to weather prediction is mentioned that is solely based on averages. I'd suggest considering other sources or more specific information if you are looking for an answer in relation to forecasting weather using average temperatures or other meteorological data.
Evaluation: 

    Rating: 9.
    The assistant correctly identifies the methods mentioned in the question (Moving Average and Exponential Smoothing) but does not directly state they are used for predicting weather based on averages, as the question asks. Instead, it provides a more detailed explanation of how these techniques can be applied to different types of time series forecasting, including weather forecasts. The assistant also suggests considering other sources or specific information if further detail is required. However, the response could benefit from a clearer statement about which methods specifically are used for weather forecasting with averages.
_________________________________________________________________________________________________________________________
Output:  Based on the information provided, Russian Blues and British Blues are different cat breeds. While both have a blue coat, Russian Blues have a dense, fine undercoat and guard hairs with silver tips, whereas British Blues do not have specific coat requirements as they come in various patterns and colors. Additionally, Russian Blues may have a slightly duller tail with almost unnoticeable stripes, which is not typical of British Blues.
Evaluation: 
    
Rating: 8.
The response is helpful as it clearly explains the differences between the two breeds. It is accurate in that it provides factual information on their coat characteristics and differences. While there could be more depth regarding their origins or temperaments, the response still offers a good level of detail for a simple user question. The creativity factor is low, but that’s not a requirement as the main goal here is to provide factually accurate and helpful information. Overall, it provides a solid response with relevant and accurate information.

This response rates an 8 due to its helpfulness, relevance, accuracy, and depth while still being brief enough for a user question. It offers more detail than simply stating 'they are different breeds' or providing vague or incorrect responses.
_________________________________________________________________________________________________________________________
Output: The prime minister is the head of government in Canada and is responsible for advising the Crown on how to exercise executive powers. The prime minister, supported by the Office of the Prime Minister, controls appointments to many key positions in the Canadian system of governance, including the governor general, members of the Cabinet, justices of the Supreme Court, senators, heads of crown corporations, ambassadors to foreign countries, and approximately 3,100 other positions. The prime minister also plays a prominent role in the legislative process, with most bills put before parliament originating in the Cabinet, and leads the Canadian Armed Forces.
Evaluation: 

    Rating: 10.

    Explanation: The response accurately describes the roles and responsibilities of the prime minister in Canada, covering aspects such as executive powers, cabinet appointments, legislative process, and armed forces command. The level of detail and depth provided is commendable. Moreover, the language used is clear, concise, and relevant to the question. Overall, an excellent response from the AI assistant.
_________________________________________________________________________________________________________________________
Output: The Latin phrase "parens patriae" refers to the power of a state to intervene on behalf of an individual who lacks the capacity to make decisions or protect themselves due to minority, mental incapacity, or other reasons. It is often used in the context of guardianship and wardship laws. Therefore, it is used as a legal concept for the state to act as the protective parent or guardian of individuals under certain circumstances.
Evaluation: 
    
    Rating: 10. The assistant provided a clear explanation with historical background and context that was accurate and detailed, addressing the user's specific question about the meaning of "parens patriae" in Latin law.
_________________________________________________________________________________________________________________________
Output: The Dadaists were a group of artists primarily active during World War I and the post-war period. They rejected the logic, reason, and aestheticism of modern capitalist society, instead expressing nonsense, irrationality, and anti-bourgeois protest in their works. They expressed discontent with violence, war, and nationalism, and maintained political affinities with the radical left. Dada was a reaction to the horrors of World War I, and these artists sought to subvert conventional norms and values through their art.
Evaluation: 

    Rating: 10.

    Explanation:
    The response is both helpful and accurate in answering the question about who the Dadaists were and what they did. It provides an informative and detailed explanation, covering the historical context, artistic movement, political context, and goals of the Dada artists. The response also demonstrates a good understanding of the subject matter, as it explains the reasons behind the Dadaists' rejection of reason and logic in art, and their motivations for subverting conventional norms and values through their works. Overall, the response provides an accurate, comprehensive, and insightful answer that would be useful to anyone seeking to understand the Dada movement.
_________________________________________________________________________________________________________________________
Output: I don't know. The context provided does not mention anything about Cinderella and her sisters.
Evaluation: 

    Rating: 0.

    The response is irrelevant to the question as it neither provides an answer nor attempts to provide one based on the available information. Therefore, it earns a rating of 0. The assistant should have acknowledged that it cannot determine the number of sisters Cinderella has from the provided context.
_________________________________________________________________________________________________________________________
Output:     I don't know. The information provided in this context is about a specific animal with physical characteristics and doesn't mention anything related to the origin of the saying "hakuna matata."
Evaluation: 

    Rating: 1

    This response from the AI assistant was rated as a 1, as it did not provide any information about where or when the phrase "hakuna matata" originated. The assistant acknowledged that it didn't have the necessary knowledge to answer the question.
_________________________________________________________________________________________________________________________
Output: The President of the United States is the head of the executive branch of the federal government and the commander-in-chief of the military. The President is responsible for enforcing the laws and executing the policies of the government, as well as negotiating treaties, appointing federal officers with the advice and consent of the Senate, and granting pardons. However, in the context provided, the question pertains to the role of the prime minister in a constitutional monarchy, which is different from the presidential system in the United States.
Evaluation: 

    Rating: 9

    The response provides an accurate and detailed explanation of the roles and responsibilities of the President of the United States. However, the user's question was about a completely different topic - the role of a president in a constitutional monarchy. Despite that, the assistant provided a well-researched answer regarding the functions of the President of the US. The response is clear, easy to understand and shows a good level of detail. However, it fails to address the user's question and provide an accurate answer for that specific context. Therefore, I deduct 1 point for the irrelevant response.
_________________________________________________________________________________________________________________________
Output: The term 'tabula rasa' comes from Latin and means "blank slate." It refers to the belief that people are born without pre-existing knowledge or innate ideas and that their minds are blank slates waiting to be written upon by experience. The phrase was popularized by John Locke, an English philosopher, but it actually predates him and can be found in various sources before his time.
Evaluation: 

    Rating: 9.
    The assistant's response is highly relevant to the question as it provides a clear explanation about the origin and meaning of the term 'tabula rasa'. It accurately identifies the Latin origin and meaning, and it also explains how the phrase was popularized by John Locke but predates him. While it doesn't provide any specific sources for these historical claims, the response is detailed enough to be helpful and informative to most users. The assistant could have provided a more creative or in-depth answer by exploring the philosophical implications of the term, but overall this response effectively answers the user question.
_________________________________________________________________________________________________________________________
Output: The provided context discusses the Dada movement, which was a cultural and artistic response to World War I, and not related to the reasons why Japan attacked Pearl Harbor during World War II. Therefore, based on the current knowledge, I cannot answer the question as it is not addressed in the context.
Evaluation: 
    Rating: 1.

    The assistant correctly identified that the user's question was outside the scope of the provided context and did not provide a response. It is important for an assistant to acknowledge its limitations and decline to answer if it cannot provide accurate or relevant information, as in this case.
_________________________________________________________________________________________________________________________"""

# This is the output; I notice that Mistral only sometimes says "I don't know". For 3/5, it said IDK, while it answered the rest; The LLM-judge rated those that didn't answer as lower, which makes sense since it didn't answer the question